# Классификация комментариев для интернет-магазина

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. 

Для обучения имеется набор данных с разметкой о токсичности правок.

Целевая метрика качетсва модели - *F1*, со значением не меньше 0.75. 

**План выполнения проекта**

1. Изучение данных;
2. Предобраотка данных;
3. Подбор моделей;
4. Анализ времени работы и эффективности отобранных моделей;
5. Формирование итогового вывода.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Подбор-модели" data-toc-modified-id="Подбор-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подбор модели</a></span></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#CatBoost-Classifier" data-toc-modified-id="CatBoost-Classifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>CatBoost Classifier</a></span></li><li><span><a href="#LGBM-Classifier" data-toc-modified-id="LGBM-Classifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>LGBM Classifier</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Подготовка

Обновим необходимые для работы библиотеки:

In [1]:
! pip install scikit-learn --upgrade

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn in /home/jovyan/.local/lib/python3.7/site-packages (1.0)


Импортируем необходимые для работы над проектом библиотеки:

In [2]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import catboost as cb
from sklearn.metrics import f1_score

Загрузим данные:

In [3]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

### Изучение данных

Выведем на экран первые 5 строчек загруженного датасета и общую информацию о данных:

In [4]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Проверим датасет на наличие грубых дубликатов:

In [6]:
data.duplicated().sum()

0

Дубликаты не обнаружены.

###### Вывод

Всего в таблице 2 столбца со следующими типами данных: int64, object.

Каждый объект в наборе данных — это информация о одном комментарии. Известно:

 - ***text*** — текс комментария
 - ***toxic*** — разметка о токсичности комментария

Пропуски и дубликаты в дтасете не обнаружены.

### Предобработка данных

Разделим данные на признаки. Целевым признаком обозначим разметку о токсичности комментария:

In [7]:
features = data.drop('toxic', axis = 1)
target = data['toxic']

Проверим разделение признаков:

In [8]:
display(features.head())
display(target.head())

,text
0,Explanation\nWhy the edits made under my usern...
1,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It..."
3,"""\nMore\nI can't make any real suggestions on ..."
4,"You, sir, are my hero. Any chance you remember..."


0    0
1    0
2    0
3    0
4    0
Name: toxic, dtype: int64

Признаки разделены. 

В тексте комментариев наблюдаются лишние символы, мешающие его полноценной обработке. Очистим текст от лишних символов, оставив только слова состоящие из символов верхнего и/или нижнего регистров.

Напишем функцию для очистки текста:

In [9]:
def clean_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

Применим функцию и создадим дополнительный столбец в ***features*** с очищенным текстом для сравнения с исходным:

In [10]:
features['clean_text'] = features['text'].apply(str).apply(lambda x: clean_text(x))

Проверим очистку текста:

In [11]:
features.head()

,text,clean_text
0,Explanation\nWhy the edits made under my usern...,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",You sir are my hero Any chance you remember wh...


Текст успешно очищен.

Прежде чем извлечь признаки из текста, необходимо упростить его: разбить на токены и лемматизировать - привести слова к начальной форме.
Напишем функцию для демматизации текста:

In [12]:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])

Применим функцию лемматизации и запишем предобработанный признак в переменную ***preprocessed_features***:

In [13]:
# preprocessed_features = []
# for line in tqdm(features['clean_text']):
#     preprocessed_features.append(lemmatize_text(line))

In [14]:
# preprocessed_features = pd.DataFrame(
#     preprocessed_features,
#     columns = ['text'],
#     index = target.index
# )

Процесс лемматизации достаточно затратный по времени. Так как работа над проектом ведется в несколько подходов, с целью экономии времени сохраним предобработанный признак в одноименный .csv - файл:

In [15]:
# preprocessed_features.to_csv('preprocessed_features.csv', index = False)

Загрузим предобработанный признак:

In [16]:
preprocessed_features = pd.read_csv(
    '/datasets/preprocessed_features.csv',
    keep_default_na = False, na_values = ' ')
preprocessed_features.head()

,text
0,Explanation Why the edits make under my userna...
1,D aww He match this background colour I m seem...
2,Hey man I m really not try to edit war It s ju...
3,More I can t make any real suggestion on impro...
4,You sir be my hero Any chance you remember wha...


Из загруженных данных видно, что текст упрощен.

Далее разделим данные на обучающую и тестовую выборки. Размер тесовой выборки зададим равным 25% от размера исходных данных.

In [17]:
features_train, features_test, target_train, target_test = train_test_split(
    preprocessed_features,
    target,
    test_size = 0.25,
    random_state = 12345,
    stratify = target
)

Проверим размеры полученных выборок:

In [18]:
print('Размер обучающей таблицы признаков:',features_train.shape)
print('Размер обучающей таблицы с целевым признаком:',target_train.shape)
print('Размер тестовой таблицы признаков:',features_test.shape)
print('Размер тестовой таблицы с целевым признаком:',target_test.shape)

Размер обучающей таблицы признаков: (119678, 1)
Размер обучающей таблицы с целевым признаком: (119678,)
Размер тестовой таблицы признаков: (39893, 1)
Размер тестовой таблицы с целевым признаком: (39893,)


Размеры выборок соответствуют заданным параметрам.

Переведем текстовые признаки обучающей и тестовой выборок в векторные представления для обеспечения возможности применять к ним модели машинного обучения.

In [19]:
vectorizer = TfidfVectorizer(stop_words='english')
X_vec_train = vectorizer.fit_transform(features_train['text'])
X_vec_test = vectorizer.transform(features_test['text'])

###### Вывод

На данном этапе текст комментариев был подготовлен к применению в моделях машинного обучения:

 - Текст разбит на токены, очищен от лишних символов и лемматизирован;
 - Текст переведен в векторный вид.

## Подбор модели

Напишем функцию для подбора наилучших параметров модели:

In [21]:
def get_best_params(model,parameters):
    
    grid = GridSearchCV(
    model,
    parameters,
    scoring = 'f1'
    )
    
    grid.fit(X_vec_train,target_train)
    
    greed_result = pd.DataFrame(
        grid.cv_results_).sort_values(
        by = 'rank_test_score').reset_index(
        drop = True)
    
    display(greed_result[greed_result['rank_test_score']==1])

Рассмотрим модель ***Логистическая регрессия***:

In [22]:
logreg = LogisticRegression()

get_best_params(
    logreg,
    {
        'solver':['saga'],
        'tol':[0.0087],
        'C':[14],
        'random_state':[12345]
        
    }
)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_random_state,param_solver,param_tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.521011,0.03148,0.004824,0.000543,14,12345,saga,0.0087,"{'C': 14, 'random_state': 12345, 'solver': 'sa...",0.756291,0.756744,0.764432,0.770305,0.758219,0.761198,0.005407,1


В результате подбора параметров наилучшее значениее метрики ***F1 = 0.761***

Рассмотрим модель ***Дерево решений***:

In [23]:
tree = DecisionTreeClassifier()

get_best_params(
    tree,
    {
        'max_depth':[48],
        'min_samples_split':[2],
        'min_samples_leaf':[1],
        'random_state':[12345]
    }
)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,15.559097,0.105141,0.025611,0.000738,48,1,2,12345,"{'max_depth': 48, 'min_samples_leaf': 1, 'min_...",0.683649,0.69723,0.700454,0.704448,0.703617,0.697879,0.007558,1


В результате подбора параметров наилучшее значениее метрики ***F1 = 0.698***

Рассмотрим модель ***CatBoostClassifier***:

In [24]:
cbc = cb.CatBoostClassifier()

get_best_params(
    cbc,
    {
        'silent':[True],
        'max_depth':[5],
        'n_estimators':[150],
        'learning_rate':[1]   
    }
)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_silent,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,45.94077,0.86521,0.312267,0.022502,1,5,150,True,"{'learning_rate': 1, 'max_depth': 5, 'n_estima...",0.736264,0.738971,0.745513,0.753324,0.751411,0.745097,0.006683,1


В результате подбора параметров наилучшее значениее метрики ***F1 = 0.751***

Рассмотрим модель ***LGBMClassifier***

In [25]:
lgbc = lgb.LGBMClassifier()

get_best_params(
    lgbc,
    {
        'objective':['binary'],
        'max_depth':[51],
        'n_estimators':[1000],
        'learning_rate':[0.1]   
    }
)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,55.548889,2.246789,2.102765,0.048621,0.1,51,1000,binary,"{'learning_rate': 0.1, 'max_depth': 51, 'n_est...",0.764987,0.768307,0.760753,0.775716,0.765478,0.767048,0.004961,1


В результате подбора параметров наилучшее значениее метрики ***F1 = 0.767***

###### Вывод

На данном этапе были рассмотрены следующие модели, для каждой из которых с помощью ***GridSearchCV*** были подобраны наилучшие параметры и получены следующие значения ***F1*** на кросс-валидации ( расположены в порядке убывания качества ):

- ***LGBM Classifier***: 0.767;
- ***Логистическая регрессия***: 0.761;
- ***CatBoost Classifier***: 0.751;
- ***Дерево решений***: 0.698.

Из результатов подбора наихудшее качество наблюдается у ***Дерева решений***. Остальные модели показывают достаточно близкий результат - далее проанализируем их более подробно.

## Анализ моделей

### Логистическая регрессия

Обучим модель ***Логистическая регрессия*** с заданием подобранных на предыдущем этапе гиперпараметров и рассчитаем время обучения:

In [25]:
%%time

logreg = LogisticRegression(
    solver = 'saga',
    C = 14,
    tol = 0.0087,
    random_state = 12345
)

logreg.fit(X_vec_train, target_train)

CPU times: user 657 ms, sys: 3.95 ms, total: 661 ms
Wall time: 663 ms


LogisticRegression(C=14, random_state=12345, solver='saga', tol=0.0087)

Время обучения ***Логистической регрессии*** = 661 милисекунда.

Рассчитаем время предсказания моделью целевого признака:

In [26]:
%%time

logreg_predicts = logreg.predict(X_vec_test)

CPU times: user 7.15 ms, sys: 17.1 ms, total: 24.3 ms
Wall time: 29.3 ms


Время предсказаний ***Логистической регрессии*** = 24.3 милисекунды.

Рассчитаем ***F1*** модели на тестовой выборке:

In [27]:
print('F1 модели на тестовой выборке:', f1_score(target_test, logreg_predicts).round(3))

F1 модели на тестовой выборке: 0.776


### CatBoost Classifier

Обучим модель ***CatBoost Classifier*** с заданием подобранных на предыдущем этапе гиперпараметров и рассчитаем время обучения:

In [28]:
%%time

cbc = cb.CatBoostClassifier(
    silent = True,
    max_depth = 5,
    n_estimators = 150,
    learning_rate = 1
)
cbc.fit(X_vec_train, target_train)

CPU times: user 4min 28s, sys: 4.57 s, total: 4min 32s
Wall time: 54.3 s


Время обучения ***CatBoost Classifier***  = 4 минуты 32 секунды.

Рассчитаем время предсказания моделью целевого признака:

In [29]:
%%time

cbc_predicts = cbc.predict(X_vec_test)

CPU times: user 858 ms, sys: 403 ms, total: 1.26 s
Wall time: 375 ms


Время предсказаний ***CatBoost Classifier***  = 1.26 секунд.

Рассчитаем ***F1*** модели на тестовой выборке:

In [30]:
print('F1 модели на тестовой выборке:',f1_score(target_test, cbc_predicts).round(3))

F1 модели на тестовой выборке: 0.758


### LGBM Classifier

Обучим модель ***LGBM Classifier*** с заданием подобранных на предыдущем этапе гиперпараметров и рассчитаем время обучения:

In [31]:
%%time

lgbc = lgb.LGBMClassifier(
    objective = 'binary',
    max_depth = 51,
    n_estimators = 1000,
    learning_rate = 0.1
)
lgbc.fit(X_vec_train, target_train)

CPU times: user 7min 26s, sys: 37.4 s, total: 8min 3s
Wall time: 1min 3s


LGBMClassifier(max_depth=51, n_estimators=1000, objective='binary')

Время обучения ***LGBM Classifier***  = 8 минут 3 секунды.

Рассчитаем время предсказания моделью целевого признака:

In [32]:
%%time

lgbc_predicts = lgbc.predict(X_vec_test)

CPU times: user 24.9 s, sys: 152 ms, total: 25.1 s
Wall time: 3.28 s


Время предсказаний ***LGBM Classifier***  = 25.1 секундa.

Рассчитаем ***F1*** модели на тестовой выборке:

In [33]:
print('F1 модели на тестовой выборке:',f1_score(target_test, lgbc_predicts).round(3))

F1 модели на тестовой выборке: 0.781


###### Вывод

Сформируем итоговую таблицу:

In [46]:
results = pd.DataFrame(
    {
        'Модель':['Логистическая регрессия', 'CatBoost Classifier','LightGBM Classifier'],
        'F1':[0.776, 0.758, 0.781],
        'Время обучения, сек': [0.661, 272, 483],
        'Время предсказаний, сек': [0.024, 1.26, 25.1]
    }
)

results.sort_values(
    by = 'F1',
    ascending = False).set_index(
    pd.Index([1, 2, 3]))

,Модель,F1,"Время обучения, сек","Время предсказаний, сек"
1,LightGBM Classifier,0.781,483.000,25.100
2,Логистическая регрессия,0.776,0.661,0.024
3,CatBoost Classifier,0.758,272.000,1.260


## Общий вывод

В ходе данного проекта данные о комментариях пользователей интернет-магазина «Викишоп» изучены, предобработаны и разделены на две выборки: обучающую и тестовую в соотношении ***75% : 25%***.

Выборки были использованы для подбора, обучения и тестирования моделей: ***Логистическая регрессия***, ***Дерево решений***, ***CatBoost Classifier***, ***LightGBM Classifier***.

В соответствии с условиями задачи целевой метрикой качества моделей было выбрано ***F1*** со значением не менее 0.75.

Для подбора гиперпараметров, а также предварительной оценки качества моделей методом кросс-валидации был применен ***GridSearchCV***.

В результате подбора моделей были выбраны три наиболее перспективных:

 - ***LGBM Classifier***;
 - ***Логистическая регрессия***;
 - ***CatBoost Classifier***.
  
которые были протестированы на скорость обучения и качество предсказаний на тестовой выборке.

Наилучшей с точки зрения метрики качества оказалась модель ***LightGBM Regressor***, с ***F1*** на тестовой выборке = ***0.781*** и с ***8 минутами и 7 секундами***, затраченными на обучение.

Наилучшей с точки зрения времени времени обучения оказалась модель ***Логистическая регрессия***, с ***F1*** на тестовой выборке = ***0.776*** и с ***661 милисекундой***, затраченной на обучение.

Так как во время теста ***Логистическая регрессия*** показала наилучший результат по времени обучения и позволила получить метрику качества ***F1***, незначительно отличающуюся от ***LightGBM Regressor***, интернет-магазину «Викишоп» рекомендуется применить ***Логистическую регрессию*** для запуска инструмента по поиску и отправки на модерацию токсичных комментриев.
